In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:

data_path = r"D:\displaced prediction\data\###.xlsx"
df = pd.read_excel(data_path, index_col=0)
df.fillna(0, inplace=True)

features = df[['dis','Average_22', 'R0', 'Diff_45']]#'Average_22', 'R0', 'Diff_45'
target = df['dis']

def normalization(data):
    mean = data.mean()
    scaled_data = (data - mean) / data.std()
    return mean, data.std(), scaled_data

def denormalization(data, mean, std):
    return data * std + mean

input_mean, input_std, input_data = normalization(features)
output_mean, output_std, output_data = normalization(target)

timestep = 3

def create_time_series(data, timestep, output_data):
    X, y = [], []
    for i in range(len(data) - timestep):
        X.append(data.iloc[i:(i + timestep)].values)
        y.append(output_data.iloc[i + timestep])
    return np.array(X), np.array(y)

X, y = create_time_series(input_data, timestep, output_data)
y = y.reshape(-1, 1)

train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))


model = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.3,
    learning_rate=0.01,
    max_depth=7,  
    alpha=10,
    n_estimators=300,  
    subsample=0.8, 
    min_child_weight=1,
    reg_alpha=0.1,  
    reg_lambda=0.1  
)

model.fit(X_train_reshaped, y_train)


y_train_pred = model.predict(X_train_reshaped)
y_test_pred = model.predict(X_test_reshaped)

y_train_pred_denorm = denormalization(y_train_pred, output_mean, output_std)
y_test_pred_denorm = denormalization(y_test_pred, output_mean, output_std)
y_test_denorm = denormalization(y_test, output_mean, output_std)

plt.figure(figsize=(14, 7))
plt.plot(y_test_denorm, color="r", label="Original")
plt.plot(y_test_pred_denorm, color="b", label="Prediction")
plt.legend()
plt.show()


In [ ]:

true = y_test_denorm
pre = y_test_pred_denorm
mse = np.sum((true-pre) ** 2) / len(pre)
rmse = np.sqrt(mse)
mae = np.sum(np.absolute(true-pre)) / len(pre)
r2 = 1-mse/ np.var(pre)
mape = np.mean(np.abs((pre - true) / true))
print(" mae:",mae,"mse:",mse," rmse:",rmse," r2:",r2," mape:",mape)
